## Imports

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=1
%matplotlib inline

In [ ]:
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
import tensorflow as tf
from data.dataset import Dataset
import data.data_selector as ds
import utils.data_processing as dp
import utils.plot_functions as pf
import analysis.analysis_picker as ap

In [ ]:
def get_activation_function():
  return tf.identity

In [ ]:
class lambda_params(object):
  def __init__(self):
    self.model_type = "lambda"
    self.model_name = "lambda_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False
    self.activation_function = get_activation_function()

class mlp_params(object):
  def __init__(self):
    self.model_type = "mlp"
    self.model_name = "mlp_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

class lca_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_768_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

class ae_deep_params(object):
  def __init__(self):
    self.model_type = "ae"
    self.model_name = "ae_deep_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

params_list = [lca_params()]
for params in params_list:
  params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)
  
analyzer_list = [ap.get_analyzer(params.model_type) for params in params_list]

In [ ]:
for analyzer, params in zip(analyzer_list, params_list):
  analyzer.setup(params)
  if(hasattr(params, "activation_function")):
    analyzer.model_params.activation_function = params.activation_function
  analyzer.setup_model(analyzer.model_params)
  analyzer.load_analysis(save_info=params.save_info)
  analyzer.model_name = params.model_name

In [ ]:
for analyzer in analyzer_list:
  if(analyzer.analysis_params.model_type.lower() != "lca"):
    pre_images = np.stack([analyzer.neuron_vis_output["optimal_stims"][target_id][-1].reshape(28,28)
      for target_id in range(len(analyzer.analysis_params.neuron_vis_targets))], axis=0)
    pre_image_fig = pf.plot_weights(pre_images, title=analyzer.model_name+" pre-images", figsize=(4,8))
    pre_image_fig.savefig(analyzer.analysis_out_dir+"/vis/pre_images.png",
        transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
for analyzer in analyzer_list:
  if(analyzer.analysis_params.model_type.lower() == "lca"):
    available_indices = np.array(range(analyzer.model.get_num_latent()))
  else:
    available_indices = np.array([1,8,9])

In [ ]:
target_neuron_idx = 0
step_idx = -1

for analyzer in analyzer_list:
  analyzer.bf_id0 = target_neuron_idx
  if(analyzer.analysis_params.model_type.lower() == "lca"):
    analyzer.bf0 = analyzer.bf_stats["basis_functions"][analyzer.bf_id0]
  else:  
    analyzer.bf0 = analyzer.neuron_vis_output["optimal_stims"][analyzer.bf_id0][step_idx]
  analyzer.bf0 = analyzer.bf0.reshape(np.prod(analyzer.model.get_input_shape()[1:]))
  analyzer.bf0 = analyzer.bf0 / np.linalg.norm(analyzer.bf0)
  
  fig, axes = plt.subplots(1, 2, figsize=(10,4))
  
  ax = pf.clear_axis(axes[0])
  ax.imshow(analyzer.bf0.reshape(28, 28), cmap="Greys_r")#, vmin=0.0, vmax=1.0)
  ax.set_title("Optimal\ninput image")
  
  if(analyzer.analysis_params.model_type.lower() != "lca"):
    axes[1].plot(analyzer.neuron_vis_output["loss"][analyzer.bf_id0])
    axes[1].set_title("Optimization loss")
  
  plt.show()

In [ ]:
def find_orth_vect(matrix):
  rand_vect = np.random.rand(matrix.shape[0], 1)
  new_matrix = np.hstack((matrix, rand_vect))
  candidate_vect = np.zeros(matrix.shape[1]+1)
  candidate_vect[-1] = 1
  orth_vect = np.linalg.lstsq(new_matrix.T, candidate_vect, rcond=None)[0] # [0] indexes lst-sqrs solution
  orth_vect = np.squeeze((orth_vect / np.linalg.norm(orth_vect)).T) 
  return orth_vect

In [ ]:
def get_rand_vectors(bf0, num_orth_directions):
  rand_vectors = bf0.T[:,None] # matrix of alternate vectors
  for orth_idx in range(num_orth_directions):
    tmp_bf1 = find_orth_vect(rand_vectors)
    rand_vectors = np.append(rand_vectors, tmp_bf1[:,None], axis=1)
  return rand_vectors.T[1:, :] # [num_vectors, vector_length]

In [ ]:
def get_alt_vectors(bf0, bf1s):
  alt_vectors = bf0.T[:,None] # matrix of alternate vectors
  for tmp_bf1 in bf1s:
    tmp_bf1 = np.squeeze((tmp_bf1 / np.linalg.norm(tmp_bf1)).T)
    alt_vectors = np.append(alt_vectors, tmp_bf1[:,None], axis=1)
  return alt_vectors.T[1:, :] # [num_vectors, vector_length]

In [ ]:
def get_norm_activity(analyzer, alt_vectors, num_imgs):
  num_alt_vectors = alt_vectors.shape[0]
  # Construct point dataset
  x_pts = np.linspace(-0.5, 3.5, int(np.sqrt(num_imgs)))
  y_pts = np.linspace(-2.0, 2.0, int(np.sqrt(num_imgs)))
  X_mesh, Y_mesh = np.meshgrid(x_pts, y_pts)
  proj_datapoints = np.stack([X_mesh.reshape(num_imgs), Y_mesh.reshape(num_imgs)], axis=1)
  
  out_dict = {
    "norm_activity": [],
    "proj_neuron0": [],
    "proj_neuron1": [],
    "proj_v": [],
    "v": [],
    "proj_datapoints": proj_datapoints,
    "X_mesh": X_mesh,
    "Y_mesh": Y_mesh}
  
  activation_list = []
  for alt_vector_idx in range(num_alt_vectors):
    alt_vector = alt_vectors[alt_vector_idx, :]
    proj_matrix, v = dp.bf_projections(analyzer.bf0, alt_vector)
    out_dict["proj_neuron0"].append(np.dot(proj_matrix, analyzer.bf0).T) #project
    out_dict["proj_neuron1"].append(np.dot(proj_matrix, alt_vector).T) #project
    out_dict["proj_v"].append(np.dot(proj_matrix, v).T) #project
    out_dict["v"].append(v)
    
    datapoints = np.stack([np.dot(proj_matrix.T, proj_datapoints[data_id,:])
      for data_id in range(num_imgs)], axis=0) #inject
    datapoints = dp.reshape_data(datapoints, flatten=False)[0]
    datapoints = {"test": Dataset(datapoints, lbls=None, ignore_lbls=None, rand_state=analyzer.rand_state)}
    # preprocess_dataset should include rescaling to be between (0,1) for mnist
    #datapoints = analyzer.model.preprocess_dataset(datapoints, params=analyzer.model_params)
    datapoints = analyzer.model.reshape_dataset(datapoints, analyzer.model_params)
    #datapoints["test"].images = (
    #  (datapoints["test"].images - np.min(datapoints["test"].images, axis=1, keepdims=True))
    #  / (np.max(datapoints["test"].images, axis=1, keepdims=True) - np.min(datapoints["test"].images,
    #  axis=1, keepdims=True)))
    datapoints["test"].images /= np.max(np.abs(datapoints["test"].images))
    #datapoints["test"].images *= analyzer.analysis_params.input_scale*3
    activations = analyzer.compute_activations(datapoints["test"].images)#, batch_size=num_imgs//16)
    activations = activations[:, analyzer.bf_id0]
    activity_max = np.amax(np.abs(activations))
    out_dict["norm_activity"].append(activations / (activity_max + 0.00001)) # Rescale between -1 and 1
  
  return out_dict

In [ ]:
analyzer = analyzer_list[0]
step_idx = -1
num_imgs = int(228**2)

num_neurons = 3
num_orth_directions = 3

all_neuron_indices = np.random.choice(available_indices, num_orth_directions+num_neurons, replace=False)#range(analyzer.model.get_num_latent())
#all_neuron_indices = np.random.choice(available_indices, num_neurons, replace=False)#range(analyzer.model.get_num_latent())

target_neuron_indices = all_neuron_indices[:num_neurons]

num_plots_y = num_neurons + 1 # extra dimension for example image
num_plots_x = num_orth_directions + 1 # extra dimension for example image

gs0 = gridspec.GridSpec(num_plots_y, num_plots_x, wspace=0.1, hspace=0.1)
fig = plt.figure(figsize=(28, 28))
cmap = plt.get_cmap('viridis')
orth_vectors = []
target_vectors = []

for neuron_loop_index, neuron_id in enumerate(target_neuron_indices): # rows
  # Get target neuron pre-image
  analyzer.bf_id0 = neuron_id
  
  if(analyzer.analysis_params.model_type.lower() == "lca"):
    analyzer.bf0 = analyzer.bf_stats["basis_functions"][analyzer.bf_id0]
  else:
    analyzer.bf0 = analyzer.neuron_vis_output["optimal_stims"][analyzer.bf_id0][step_idx] # shape=[28,28,1]

  target_vectors.append(analyzer.bf0)
  analyzer.bf0 = analyzer.bf0.reshape(np.prod(analyzer.model.get_input_shape()[1:])) # shape=[784]
  #analyzer.bf0 = (analyzer.bf0 - np.min(analyzer.bf0)) / (np.max(analyzer.bf0) - np.min(analyzer.bf0)) # map to data range (0,1)
  analyzer.bf0 = analyzer.bf0 / np.linalg.norm(analyzer.bf0) # normalize length

  #TODO: I think this is wrong - don't need to compute every for neuron_idx inside loop
  if orth_neuron_indices is None:
    if(analyzer.analysis_params.model_type.lower() == "lca"):
      min_angle = 50
      max_angle = 80
      vectors = np.squeeze(np.argwhere(np.logical_and(analyzer.neuron_angles[analyzer.bf_id0, :] < max_angle,
        analyzer.neuron_angles[analyzer.bf_id0, :] > min_angle)))
      vector_ids = np.random.choice(vectors, num_orth_directions, replace=False)
      alt_stims =  [analyzer.bf_stats["basis_functions"][vector_id].reshape(28*28)
        for vector_id in vector_ids]
      comparison_vectors = get_alt_vectors(analyzer.bf0, alt_stims)
    else:
      comparison_vectors = get_rand_vectors(analyzer.bf0, num_orth_directions)
  else:
    alt_stims =  [analyzer.neuron_vis_output["optimal_stims"][orth_neuron_idx][step_idx]
      for orth_neuron_idx in orth_neuron_indices]
    comparison_vectors = get_alt_vectors(analyzer.bf0, alt_stims)

  out_dict = get_norm_activity(analyzer, comparison_vectors, num_imgs)
  analyzer.X_mesh = out_dict["X_mesh"]
  analyzer.Y_mesh = out_dict["Y_mesh"]

  for orth_loop_index in range(num_orth_directions): # columns
      analyzer.norm_activity = out_dict["norm_activity"][orth_loop_index]
      analyzer.norm_activity = analyzer.norm_activity.reshape(int(np.sqrt(num_imgs)), int(np.sqrt(num_imgs)))
      analyzer.proj_neuron0 = out_dict["proj_neuron0"][orth_loop_index]
      analyzer.proj_neuron1 = out_dict["proj_neuron1"][orth_loop_index]
      analyzer.proj_v = out_dict["proj_v"][orth_loop_index]
      orth_vectors.append(out_dict["v"][orth_loop_index])
      
      curve_plot_y_idx = neuron_loop_index + 1
      curve_plot_x_idx = orth_loop_index + 1
      curve_ax = pf.clear_axis(fig.add_subplot(gs0[curve_plot_y_idx, curve_plot_x_idx]))

      # NOTE: each subplot has a renormalized color scale
      # TODO: Add scale bar like in the lca inference plots
      vmin = np.floor(np.min(analyzer.norm_activity))
      vmax = np.ceil(np.max(analyzer.norm_activity))

      levels = 5
      contsf = curve_ax.contourf(analyzer.X_mesh, analyzer.Y_mesh, analyzer.norm_activity,
        levels=levels, vmin=vmin, vmax=vmax, alpha=1.0, antialiased=True, cmap=cmap)

      curve_ax.arrow(0, 0, analyzer.proj_neuron0[0].item(), analyzer.proj_neuron0[1].item(),
        width=0.05, head_width=0.15, head_length=0.15, fc='r', ec='r')
      curve_ax.arrow(0, 0, analyzer.proj_neuron1[0].item(), analyzer.proj_neuron1[1].item(),
        width=0.05, head_width=0.15, head_length=0.15, fc='w', ec='w')
      curve_ax.arrow(0, 0, analyzer.proj_v[0].item(), analyzer.proj_v[1].item(),
        width=0.05, head_width=0.15, head_length=0.15, fc='k', ec='k')

      curve_ax.set_xlim([-0.5, 3.5])
      curve_ax.set_ylim([-2, 2.0])
      #curve_ax.set_aspect("equal")

for plot_y_id in range(num_plots_y):
  for plot_x_id in range(num_plots_x):
      if plot_y_id > 0 and plot_x_id == 0:
        bf_ax = pf.clear_axis(fig.add_subplot(gs0[plot_y_id, plot_x_id]))
        bf_ax.imshow(target_vectors[plot_y_id-1].reshape((28,28)), cmap="Greys_r")
        if plot_y_id == 1:
          bf_ax.set_title("Target vectors", color="r", fontsize=16)
      if plot_y_id == 0 and plot_x_id > 0:
        #comparison_img = comparison_vectors[plot_x_id-1, :].reshape(28,28)
        orth_img = orth_vectors[plot_x_id-1].reshape(28,28)
        orth_ax = pf.clear_axis(fig.add_subplot(gs0[plot_y_id, plot_x_id]))
        orth_ax.imshow(orth_img, cmap="Greys_r")
        if plot_x_id == 1:
          #orth_ax.set_ylabel("Orthogonal vectors", color="k", fontsize=16)
          orth_ax.set_title("Orthogonal vectors", color="k", fontsize=16)

plt.show()

In [ ]:
"""
* Compute a unit vector that is in the same plane as a given basis function pair (B1,B2) and is orthogonal to B1, where B1 is the target basis for comparison and B2 is selected from all other bases.
* Construct a line of data points in this plane
* Project the data points into image space, compute activations, plot activations
"""
for analyzer in analyzer_list:
  analyzer.pop_num_imgs = 100
  
  orthogonal_list = [idx for idx in range(analyzer.bf_stats["num_outputs"])]
  num_orthogonal = len(orthogonal_list)
  
  pop_x_pts = np.linspace(-2.0, 2.0, int(analyzer.pop_num_imgs))
  pop_y_pts = np.linspace(-2.0, 2.0, int(analyzer.pop_num_imgs))
  pop_X, pop_Y = np.meshgrid(pop_x_pts, pop_y_pts)
  pop_proj_datapoints = np.stack([pop_X.reshape(analyzer.pop_num_imgs**2), pop_Y.reshape(analyzer.pop_num_imgs**2)], axis=1) # construct a grid
  
  #x_target = pop_x_pts[int(6*analyzer.pop_num_imgs/8)] # find a location to take a slice
  x_target = pop_x_pts[int(0.75*analyzer.pop_num_imgs)] # find a location to take a slice
  
  slice_indices = np.where(pop_proj_datapoints[:,0]==x_target)[0]
  analyzer.pop_proj_datapoints = pop_proj_datapoints[slice_indices,:] # slice grid
  
  pop_datapoints = [None,]*num_orthogonal
  #pop_proj_neurons = [None,]*num_orthogonal
  for pop_idx, tmp_bf_id1 in enumerate(orthogonal_list):
    tmp_bf1 = analyzer.bf_stats["basis_functions"][tmp_bf_id1].reshape((analyzer.model_params.num_pixels))
    tmp_bf1 /= np.linalg.norm(tmp_bf1)
    tmp_proj_matrix, v = analyzer.bf_projections(analyzer.bf0, tmp_bf1) 
    pop_datapoints[pop_idx] = np.dot(pop_proj_datapoints, tmp_proj_matrix)#[slice_indices,:]
  
  pop_datapoints = np.reshape(np.stack(pop_datapoints, axis=0),
    [num_orthogonal*analyzer.pop_num_imgs, analyzer.model_params.num_pixels])
  
  pop_datapoints = dp.reshape_data(pop_datapoints, flatten=False)[0]
  analyzer.pop_datapoints = {"test": Dataset(pop_datapoints, lbls=None, ignore_lbls=None, rand_state=analyzer.rand_state)}
  analyzer.pop_datapoints = analyzer.model.preprocess_dataset(analyzer.pop_datapoints,
    params={"whiten_data":analyzer.model_params.whiten_data,
    "whiten_method":analyzer.model_params.whiten_method})
  analyzer.pop_datapoints = analyzer.model.reshape_dataset(analyzer.pop_datapoints, analyzer.model_params)
  analyzer.pop_datapoints["test"].images /= np.max(np.abs(analyzer.pop_datapoints["test"].images))
  analyzer.pop_datapoints["test"].images *= analyzer.analysis_params.input_scale

In [ ]:
for analyzer in analyzer_list:
  pop_activations = analyzer.compute_activations(analyzer.pop_datapoints["test"].images)[:, analyzer.bf_id0]
  pop_activations = pop_activations.reshape([num_orthogonal, pop_num_imgs])
  analyzer.pop_norm_activity = pop_activations / np.amax(np.abs(pop_activations))

In [ ]:
"""
* Construct the set of unit-length bases that are orthogonal to B0 (there should be B0.size-1 of them)
* Construct a line of data points in each plane defined by B0 and a given orthogonal basis
* Project the data points into image space, compute activations, plot activations
"""
for analyzer in analyzer_list:
  analyzer.rand_pop_num_imgs = 100
  analyzer.rand_num_orthogonal = analyzer.bf_stats["num_inputs"]-1
  
  pop_x_pts = np.linspace(-2.0, 2.0, int(analyzer.rand_pop_num_imgs))
  pop_y_pts = np.linspace(-2.0, 2.0, int(analyzer.rand_pop_num_imgs))
  pop_X, pop_Y = np.meshgrid(pop_x_pts, pop_y_pts)
  analyzer.rand_pop_proj_datapoints = np.stack([analyzer.pop_X.reshape(analyzer.rand_pop_num_imgs**2),
    analyzer.pop_Y.reshape(analyzer.rand_pop_num_imgs**2)], axis=1) # construct a grid
  
  #x_target = pop_x_pts[int(6*analyzer.rand_pop_num_imgs/8)] # find a location to take a slice
  x_target = pop_x_pts[int(0.25*analyzer.rand_pop_num_imgs)] # find a location to take a slice
  
  slice_indices = np.where(analyzer.rand_pop_proj_datapoints[:,0]==x_target)[0]
  analyzer.rand_pop_proj_datapoints = analyzer.rand_pop_proj_datapoints[slice_indices,:] # slice grid
  
  analyzer.rand_pop_datapoints = [None,]*analyzer.rand_num_orthogonal
  for pop_idx in range(analyzer.rand_num_orthogonal):
    v = orth_col_matrix[:, pop_idx]
    tmp_proj_matrix = np.stack([analyzer.bf0, v], axis=0)
    analyzer.rand_pop_datapoints[pop_idx] = np.dot(analyzer.rand_pop_proj_datapoints,
      tmp_proj_matrix)#[slice_indices,:]

  analyzer.rand_pop_datapoints = np.reshape(np.stack(analyzer.rand_pop_datapoints, axis=0),
    [analyzer.rand_num_orthogonal*analyzer.rand_pop_num_imgs, analyzer.model_params.num_pixels])

  analyzer.rand_pop_datapoints = dp.reshape_data(analyzer.rand_pop_datapoints, flatten=False)[0]
  analyzer.rand_pop_datapoints = {"test": Dataset(analyzer.rand_pop_datapoints, lbls=None, ignore_lbls=None, rand_state=analyzer.rand_state)}
  analyzer.rand_pop_datapoints = analyzer.model.preprocess_dataset(analyzer.rand_pop_datapoints,
    params={"whiten_data":False, "whiten_method":None})
  analyzer.rand_pop_datapoints = analyzer.model.reshape_dataset(analyzer.rand_pop_datapoints, analyzer.model_params)
  analyzer.rand_pop_datapoints["test"].images /= np.max(np.abs(analyzer.rand_pop_datapoints["test"].images))
  analyzer.rand_pop_datapoints["test"].images *= analyzer.analysis_params.input_scale

In [ ]:
analyzer.analysis_params.input_scale

In [ ]:
for analyzer in analyzer_list:
  rand_pop_activations = analyzer.compute_activations(analyzer.rand_pop_datapoints["test"].images)[:, analyzer.bf_id0]
  rand_pop_activations = rand_pop_activations.reshape([analyzer.rand_num_orthogonal, analyzer.rand_pop_num_imgs])
  analyzer.rand_pop_norm_activity = rand_pop_activations / np.amax(np.abs(rand_pop_activations))

In [ ]:
print(np.min(analyzer.norm_activity))
print(np.max(analyzer.norm_activity))
print(analyzer.norm_activity.shape)

In [ ]:
for analyzer in analyzer_list:
  analyzer.bf_coeffs = [
    np.polynomial.polynomial.polyfit(analyzer.pop_proj_datapoints[:,1], analyzer.pop_norm_activity[orthog_idx,:], deg=2)
    for orthog_idx in range(num_orthogonal)]
  analyzer.bf_fits = [
    np.polynomial.polynomial.polyval(analyzer.pop_proj_datapoints[:,1], coeff)
    for coeff in analyzer.bf_coeffs]
  analyzer.bf_curvatures = [np.polyder(fit, m=2) for fit in analyzer.bf_fits]
  
  analyzer.rand_coeffs = [np.polynomial.polynomial.polyfit(analyzer.rand_pop_proj_datapoints[:,1],
    analyzer.rand_pop_norm_activity[orthog_idx,:], deg=2) for orthog_idx in range(analyzer.rand_num_orthogonal)]
  analyzer.rand_fits = [np.polynomial.polynomial.polyval(analyzer.rand_pop_proj_datapoints[:,1], coeff)
    for coeff in analyzer.rand_coeffs]
  analyzer.rand_curvatures = [np.polyder(fit, m=2) for fit in analyzer.rand_fits]

In [ ]:
analyzer_idx = 0

bf_curvatures = np.stack(analyer_list[analyzer_idx].bf_coeffs, axis=0)[:,2]
rand_curvatures = np.stack(analyzer_list[analyzer_idx].rand_coeffs, axis=0)[:,2]

num_bins = 100
bins = np.linspace(-0.2, 0.2, num_bins)
bf_hist, bin_edges = np.histogram(bf_curvatures.flatten(), bins)
rand_hist, _ = np.histogram(rand_curvatures.flatten(), bins)
bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
bin_centers = bin_left + (bin_right - bin_left)/2

fig, ax = plt.subplots(1, figsize=(16,9))

ax.bar(bin_centers, rand_hist, width=0.0022, log=False, color="g", alpha=0.5, align="center", label="Random Projection")
ax.bar(bin_centers, bf_hist, width=0.0022, log=False, color="r", alpha=0.5, align="center", label="BF Projection")

ax.set_xticks(bin_left, minor=True)
ax.set_xticks(bin_left[::15], minor=False)
ax.xaxis.set_major_formatter(FormatStrFormatter("%0.3f"))
for tick in ax.xaxis.get_major_ticks():
  tick.label.set_fontsize(24) 
for tick in ax.yaxis.get_major_ticks():
  tick.label.set_fontsize(24) 

ax.set_title("Histogram of Curvatures", fontsize=32)
ax.set_xlabel("Curvature", fontsize=32)
ax.set_ylabel("Count", fontsize=32)
ax.legend(loc=2, fontsize=32)
fig.savefig(analyzer.analysis_out_dir+"/vis/histogram_of_curvatures_bf0id"+str(analyzer.bf_id0)+".png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)
plt.show()